In [1]:
!pip install geemap
import ee, geemap
ee.Authenticate()
ee.Initialize()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 KB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 KB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 KB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.6/99.6 KB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 KB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━

In [2]:
import folium


def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [3]:
myImage = ee.Image("LANDSAT/LT05/C01/T1_SR/LT05_015030_20100531")
vis_params = {'bands': ['B4', 'B3', 'B2'], 'min': 0.5, 'gamma': [0.95, 1.1, 1]}
map = folium.Map(location = [43.046, -76.147], zoom_start = 10)
map.add_ee_layer(myImage, vis_params, 'map')
map

In [4]:
map2 = geemap.Map(center =[43.046, -76.147], zoom = 10)
map2.addLayer(myImage, {'bands': ['B4', 'B3', 'B2']}, 'ColorIR composite')
map2

Map(center=[43.046, -76.147], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chil…

In [5]:
map3 = geemap.Map(center =[43.046, -76.147], zoom = 10)
vizParams = {'bands': ['B4', 'B3', 'B2'], 'min': 0.5, 'gamma': [0.95, 1.1, 1]}
map3.addLayer(myImage, vizParams, 'ColorIR composite')
map3

Map(center=[43.046, -76.147], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chil…

In [6]:
map4 = geemap.Map(center = [43.046, -76.147], zoom = 10)

roi = ee.Geometry.Point([-76.147, 43.046]).buffer(20000)

map4.addLayer(roi)

map4.addLayer(myImage.clip(roi))

map4

Map(center=[43.046, -76.147], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chil…

**Question 1: LT 05 means Landsat Thematic Mapper 05, C01 means Collection 01, and T1_TOA means Tier 1 Top of Atmosphere.**

**Question 2: Landsat Tirs-only 05, 015 = the WRS path, 030 = WRS row, and the final part indicates the imagery was processed on 05/31/2010.**

In [7]:
point = ee.Geometry.Point(-90.1994, 38.627)
start = ee.Date('2020-01-01')
end = ee.Date('2022-08-02')

filteredCollection = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA').filterBounds(point).filterDate(start, end).filterMetadata('CLOUD_COVER', 'less_than', 3).sort('CLOUD_COVER');

map5 = geemap.Map(center = [38.627, -90.1994], zoom = 10)

map5.addLayer(point)

map5.addLayer(filteredCollection)

print(filteredCollection.size().getInfo(), 'Images Contained in the Filtered Collection')

map5

23 Images Contained in the Filtered Collection


Map(center=[38.627, -90.1994], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chi…

In [8]:
filteredCollection = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA').filter(ee.Filter.eq('WRS_PATH', 24)).filter(ee.Filter.eq('WRS_ROW', 33)).filterDate('2014-01-01', '2015-01-01')

print('Collection: ', filteredCollection.size().getInfo(), 'images')

filteredCollection.aggregate_array('system:id').getInfo()

Collection:  22 images


['LANDSAT/LC08/C01/T1_TOA/LC08_024033_20140101',
 'LANDSAT/LC08/C01/T1_TOA/LC08_024033_20140202',
 'LANDSAT/LC08/C01/T1_TOA/LC08_024033_20140218',
 'LANDSAT/LC08/C01/T1_TOA/LC08_024033_20140306',
 'LANDSAT/LC08/C01/T1_TOA/LC08_024033_20140322',
 'LANDSAT/LC08/C01/T1_TOA/LC08_024033_20140407',
 'LANDSAT/LC08/C01/T1_TOA/LC08_024033_20140423',
 'LANDSAT/LC08/C01/T1_TOA/LC08_024033_20140509',
 'LANDSAT/LC08/C01/T1_TOA/LC08_024033_20140525',
 'LANDSAT/LC08/C01/T1_TOA/LC08_024033_20140610',
 'LANDSAT/LC08/C01/T1_TOA/LC08_024033_20140626',
 'LANDSAT/LC08/C01/T1_TOA/LC08_024033_20140712',
 'LANDSAT/LC08/C01/T1_TOA/LC08_024033_20140728',
 'LANDSAT/LC08/C01/T1_TOA/LC08_024033_20140813',
 'LANDSAT/LC08/C01/T1_TOA/LC08_024033_20140829',
 'LANDSAT/LC08/C01/T1_TOA/LC08_024033_20140914',
 'LANDSAT/LC08/C01/T1_TOA/LC08_024033_20140930',
 'LANDSAT/LC08/C01/T1_TOA/LC08_024033_20141016',
 'LANDSAT/LC08/C01/T1_TOA/LC08_024033_20141101',
 'LANDSAT/LC08/C01/T1_TOA/LC08_024033_20141117',
 'LANDSAT/LC08/C01/T

In [9]:
filteredCollection.aggregate_array('CLOUD_COVER').getInfo()

[92.08000183105469,
 76.41000366210938,
 58.400001525878906,
 0.6399999856948853,
 66.26000213623047,
 100,
 40.83000183105469,
 42.130001068115234,
 67.05999755859375,
 78.11000061035156,
 19.739999771118164,
 44.709999084472656,
 67.26000213623047,
 3.0799999237060547,
 12.279999732971191,
 23.030000686645508,
 32.36000061035156,
 1.2599999904632568,
 0.14000000059604645,
 68.93000030517578,
 41.720001220703125,
 91.41000366210938]

In [10]:
filteredCollection.aggregate_array('SUN_ELEVATION').getInfo()

[25.12160873413086,
 30.22369384765625,
 35.02796936035156,
 40.77009201049805,
 46.93415832519531,
 52.98386764526367,
 58.388126373291016,
 62.647464752197266,
 65.44874572753906,
 66.61231994628906,
 66.31148529052734,
 64.84430694580078,
 62.434505462646484,
 59.176334381103516,
 55.10920333862305,
 50.34010314941406,
 45.072235107421875,
 39.641902923583984,
 34.461341857910156,
 30.006540298461914,
 26.739017486572266,
 25.055858612060547]

In [11]:
map6 = geemap.Map(center = [38.627, -90.1994], zoom = 10)
image = ee.Image(filteredCollection.sort('CLOUD_COVER').first())
map6.addLayer(image, vizParams)
map6

Map(center=[38.627, -90.1994], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chi…

In [12]:
recentImages = filteredCollection.sort('system:time_start', False).limit(10)
recentImages.aggregate_array('system:id').getInfo()

['LANDSAT/LC08/C01/T1_TOA/LC08_024033_20141219',
 'LANDSAT/LC08/C01/T1_TOA/LC08_024033_20141203',
 'LANDSAT/LC08/C01/T1_TOA/LC08_024033_20141117',
 'LANDSAT/LC08/C01/T1_TOA/LC08_024033_20141101',
 'LANDSAT/LC08/C01/T1_TOA/LC08_024033_20141016',
 'LANDSAT/LC08/C01/T1_TOA/LC08_024033_20140930',
 'LANDSAT/LC08/C01/T1_TOA/LC08_024033_20140914',
 'LANDSAT/LC08/C01/T1_TOA/LC08_024033_20140829',
 'LANDSAT/LC08/C01/T1_TOA/LC08_024033_20140813',
 'LANDSAT/LC08/C01/T1_TOA/LC08_024033_20140728']

In [13]:
map7 = geemap.Map(center = [38.627, -90.1994], zoom = 10)
vizParams = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.5, 'gamma': [0.95, 1.1, 1]}
map7.addLayer(recentImages, vizParams)
map7

Map(center=[38.627, -90.1994], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chi…

**Question 3: Between 01/01/2014 and 01/01/2015. Cloud cover is less than 3 percent. Sun elevation is 66.612**

In [14]:
map8 = geemap.Map(center = [38.627, -90.1994], zoom = 10)

image1 = ee.Image('LANDSAT/LT05/C01/T1_TOA/LT05_015030_19880619')
image2 = ee.Image('LANDSAT/LT05/C01/T1_TOA/LT05_015030_20100531')

ndvi1 = image1.normalizedDifference(['B4', 'B3'])
ndvi2 = image2.normalizedDifference(['B4', 'B3'])

ndviDifference = ndvi2.subtract(ndvi1)

map8.addLayer(ndviDifference)

map8

Map(center=[38.627, -90.1994], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chi…

 **Question 4: High amounts of vegetation causes saturation, atmospheric disturbance, and low sensor quality.** 